In [1]:
%use kandy, multik
import generator.Generator
import generator.Point
import generator.Sector

In [14]:
import org.apache.commons.math3.distribution.PoissonDistribution
import kotlin.random.Random

val rand = Random(1337)

val A = 10
val distance = 5
val (p1, p2) = Generator.nextPariInSquare(distance, A)

val s1 = Sector(p1, rand.nextDouble(0.0, 2 * PI), PI / 2)
val s2 = Sector(p2, rand.nextDouble(0.0, 2 * PI), PI / 2)

val fovDist = distance / 2

val s1de = s1.getDirectionEndPoint(fovDist)
val s1le = s1.getLineEndsInSquare(fovDist)
val s2de = s2.getDirectionEndPoint(fovDist)
val s2le = s2.getLineEndsInSquare(fovDist)


val obstacles = HashSet<Point>()
val a = 0.1
val radius = 0.1
repeat(PoissonDistribution(a * A * A).sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

var block = false
check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
    for (p in obstacles) {
        if (i.isInCircle(p, radius)) {
            block = true
            break@check
        }
    }
}
println(s1)
println(s2)
println(s1.contains(p2))
println(s2.contains(p1))

Sector(pivot=Point(x=5.054166983725608, y=7.499413118688879), direction=320, fov=90, bu=365, bl=275)
Sector(pivot=Point(x=4.945833016274392, y=2.5005868813111203), direction=129, fov=90, bu=174, bl=84)
Sector: Sector(pivot=Point(x=5.054166983725608, y=7.499413118688879), direction=320, fov=90, bu=365, bl=275)
Other: Point(x=4.945833016274392, y=2.5005868813111203)
Angle: 268
false
Sector: Sector(pivot=Point(x=4.945833016274392, y=2.5005868813111203), direction=129, fov=90, bu=174, bl=84)
Other: Point(x=5.054166983725608, y=7.499413118688879)
Angle: 88
true


In [15]:
import org.jetbrains.letsPlot.geom.extras.arrow

val minX = min(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.x }.min(), 0.0)
val minY = min(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.y }.min(), 0.0)

val maxX = max(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.x }.max(), A.toDouble())
val maxY = max(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.y }.max(), A.toDouble())

val psize = 25
plot {

//    points {
//        x(obstacles.map { it.x }) {
//            scale = continuous(minX, maxX)
//        }
//        y(obstacles.map { it.y }) {
//            scale = continuous(minY, maxY)
//        }
//        size = psize.toDouble() * radius
//    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(minX, maxX)
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(minY, maxY)
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(minX, maxX)
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(minY, maxY)
        }
        type = LineType.DASHED
        color = Color.RED
    }
    
    for (p in listOf(
        Pair(s1, s1de),
        Pair(s2, s2de)
    )) {
        line {
            x(listOf(p.first.pivot, p.second).map { it.x }) {
                scale = continuous(minX, maxX)
            }
            y(listOf(p.first.pivot, p.second).map { it.y }) {
                scale = continuous(minY, maxY)
            }
            type = LineType.DASHED
            color = Color.GREEN
            alpha = .6
        }
    }

    for (p in listOf(
        Pair(s1, s1le.first),
        Pair(s1, s1le.second),
        Pair(s2, s2le.first),
        Pair(s2, s2le.second),
    )) {
        line {
            x(listOf(p.first.pivot, p.second).map { it.x }) {
                scale = continuous(minX, maxX)
            }
            y(listOf(p.first.pivot, p.second).map { it.y }) {
                scale = continuous(minY, maxY)
            }
            type = LineType.DASHED
            color = Color.YELLOW
            alpha = .3
        }
    }

    layout {
        size = Pair(A * psize * 2, A * psize * 2)
        style {
            axis {
//                blankAxes()
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Z8QrPd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":500.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.054166983725608,4.945833016274392],
"y":[7.499413118688879,2.5005868813111203]
},
"color":"#ee6666",
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.054166983725608,4.945833016274392],
"y":[7.499413118688879,2.5005868813111203]
},
"color":"#ee6666",
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.054166983725608,6.593459854957231],
"y":[7.499413118688879,6.2224724557857405]
},
"color":"#3ba272",
"linetype":"dashed",
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[4.945833016274392,3.6701603540361507],
"y":[2.5005868813111203,4.040930758118503]
},
"color":"#3ba272",
"linetype":"dashed",
"sampling":"none",
"alpha":0.6,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.054166983725608,7.045544813117254],
"y":[7.499413118688879,7.6849241442572165]
},
"color":"#fac858",
"linetype":"dashed",
"sampling":"none",
"alpha":0.3,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.054166983725608,5.239678009293946],
"y":[7.499413118688879,5.508035289297233]
},
"color":"#fac858",
"linetype":"dashed",
"sampling":"none",
"alpha":0.3,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[4.945833016274392,2.954608625581759],
"y":[2.5005868813111203,2.68773769191784]
},
"color":"#fac858",
"linetype":"dashed",
"sampling":"none",
"alpha":0.3,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[4.945833016274392,5.132983826881111],
"y":[2.5005868813111203,4.491811272003753]
},
"color":"#fac858",
"linetype":"dashed",
"sampling":"none",
"alpha":0.3,
"position":"identity",
"geom":"line"
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("Z8QrPd");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [2]:
import java.text.DecimalFormat

val repeats = 10000
var blocks = 0
repeat(repeats) {
    val (p1, p2) = Generator.nextPariInSquare(h, A)
    val obstacles = HashSet<Point>()
    repeat(poissonDistribution.sample()) {
        obstacles.add(Generator.nextPointInSquare(A))
    }
    var block = false
    check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
        for (p in obstacles) {
            if (i.isInCircle(p, r)) {
                block = true
                break@check
            }
        }
    }
    if (block) {
        ++blocks
    }
}
println("Probability: ${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[2], line 6, column 47: Unresolved reference: h
at Cell In[2], line 6, column 50: Unresolved reference: A
at Cell In[2], line 8, column 12: Unresolved reference: poissonDistribution
at Cell In[2], line 9, column 51: Unresolved reference: A
at Cell In[2], line 14, column 33: Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public val Int.r: RInt defined in org.jetbrains.kotlinx.multik.ndarray.data

In [3]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

val hs = listOf<Number>(0.1, 1, 4, 10) // point distance
val `as` = listOf<Number>(0.01, 0.05, 0.1, 0.3, 0.5) // obstacle density
val rs = listOf<Number>(0.05, 0.1, 1, 3) // obstacle size


println("a\th\tr\tProbability")
for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            println("$a\t$h\t$r\t${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")
        }
    }
}

a	h	r	Probability
0.01	0.1	0.05	0.00
0.01	0.1	0.1	0.00
0.01	0.1	1	0.028
0.01	0.1	3	0.206
0.01	1	0.05	0.001
0.01	1	0.1	0.003
0.01	1	1	0.04
0.01	1	3	0.229
0.01	4	0.05	0.004
0.01	4	0.1	0.003
0.01	4	1	0.115
0.01	4	3	0.363
0.01	10	0.05	0.012
0.01	10	0.1	0.019
0.01	10	1	0.21
0.01	10	3	0.47
0.05	0.1	0.05	0.003
0.05	0.1	0.1	0.002
0.05	0.1	1	0.142
0.05	0.1	3	0.659
0.05	1	0.05	0.007
0.05	1	0.1	0.01
0.05	1	1	0.212
0.05	1	3	0.71
0.05	4	0.05	0.02
0.05	4	0.1	0.04
0.05	4	1	0.39
0.05	4	3	0.886
0.05	10	0.05	0.046
0.05	10	0.1	0.10
0.05	10	1	0.673
0.05	10	3	0.97
0.1	0.1	0.05	0.002
0.1	0.1	0.1	0.003
0.1	0.1	1	0.262
0.1	0.1	3	0.876
0.1	1	0.05	0.01
0.1	1	0.1	0.024
0.1	1	1	0.393
0.1	1	3	0.936
0.1	4	0.05	0.039
0.1	4	0.1	0.093
0.1	4	1	0.67
0.1	4	3	0.988
0.1	10	0.05	0.09
0.1	10	0.1	0.18
0.1	10	1	0.897
0.1	10	3	1.00
0.3	0.1	0.05	0.005
0.3	0.1	0.1	0.014
0.3	0.1	1	0.581
0.3	0.1	3	0.995
0.3	1	0.05	0.035
0.3	1	0.1	0.066
0.3	1	1	0.749
0.3	1	3	0.999
0.3	4	0.05	0.096
0.3	4	0.1	0.228
0.3	4	1	0.965
0.3	4	3	1.00
0.3	10	0.

In [214]:
import org.apache.commons.math3.distribution.PoissonDistribution

val A = 10 // square side

val h = 1 // point distance
val a = 0.1 // obstacle density
val r = 0.3 // obstacle size

val poissonDistribution = PoissonDistribution(a * A * A)

val (p1, p2) = Generator.nextPariInSquare(h, A)
val obstacles = HashSet<Point>()
repeat(poissonDistribution.sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

val psize = 25
plot {
    
    points { 
        x(obstacles.map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(obstacles.map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        size = psize.toDouble() * r
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        type = LineType.DASHED
        color = Color.RED
    }

    layout {
        size = Pair(A * psize, A * psize)
        style { 
            axis {
                blankAxes()
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="b3j9pQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":250.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.350387563257286,6.096302500541321,1.0926554371650599,9.055118566153727,2.1818656242193377,6.804628553035297,5.024717128859306,6.811073143570087,4.218829501829326,0.2896607235726645,7.73302943410161,2.848427209715302],
"y":[7.465227078594135,5.01596508173029,2.8287960066465114,0.11275054380814997,0.3373646099401584,4.990178980212315,0.3864780850937821,4.318744100736556,5.928039795468438,7.344799156411402,3.5305908035132907,7.117550316686019]
},
"size":7.5,
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.319317406160088,4.680682593839912],
"y":[4.615245020664847,5.384754979335153]
},
"color":"#ee6666",
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[5.319317406160088,4.680682593839912],
"y":[4.615245020664847,5.384754979335153]
},
"color":"#ee6666",
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"line"
}],
"theme":{
"axis":"blank",
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("b3j9pQ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>">